In [1]:
# 全局设置
import os
import datetime as dt

import numpy as np
import pandas as pd

import QuantStudio.api as QS
fd = QS.FactorDB.FactorTools
Factorize = QS.FactorDB.Factorize

# HDB = QS.FactorDB.HDF5DB(sys_args={"主目录": "../Data/HDF5"}).connect()
HDB = QS.FactorDB.HDF5DB(config_file="../config/HDF5DBConfig.json").connect()

# RDB = QS.RiskDB.HDF5FRDB(sys_args={"主目录":"../Data/RiskData"}).connect();
RDB = QS.RiskDB.HDF5FRDB(config_file="../config/HDF5FRDBConfig.json").connect();

# 风险预算模型

对于组合 $\mathbf{w}$ 的风险度量 $\mathcal{R}\left(\mathbf{w}\right)$ 采用波动率度量：
$$
\mathcal{R}\left(\mathbf{w}\right) = \sigma\left(\mathbf{w}\right) = \sqrt{\mathbf{w}^T\cdot\mathbf{\Sigma}\cdot\mathbf{w}}
$$
其中, $\Sigma$ 是协方差矩阵, 边际风险贡献定义为 $\sigma\left(\mathbf{w}\right)$ 相对于 $\mathbf{w}$ 的梯度：
$$
\nabla_{\mathbf{w}}\sigma\left(\mathbf{w}\right) = \frac{\mathbf{\Sigma}\mathbf{w}}{\sqrt{\mathbf{w}^T\mathbf{\Sigma}\mathbf{w}}}
$$
第 i 个证券对于组合的风险贡献度定义为：
$$
\mathcal{RC}_i=w_i\cdot\frac{\left(\mathbf{\Sigma}\mathbf{w}\right)_i}{\sqrt{\mathbf{w}^T\mathbf{\Sigma}\mathbf{w}}}
$$
则可以将组合的总风险分解到各个证券上：
$$
\begin{aligned}
\sigma\left(\mathbf{w}\right) &= \mathbf{w}^T\cdot\frac{\mathbf{\Sigma}\mathbf{w}}{\sqrt{\mathbf{w}^T\mathbf{\Sigma}\mathbf{w}}}\\
&= \sum\limits_{i=1}^{n}w_i\cdot\frac{\left(\mathbf{\Sigma}\mathbf{w}\right)_i}{\sqrt{\mathbf{w}^T\mathbf{\Sigma}\mathbf{w}}}\\
&= \sum\limits_{i=1}^{n}\mathcal{RC}_i\\
\end{aligned}
$$
对于给定的风险预算 $\mathbf{b}$, 定义风险预算组合为：
$$
\left\{ \begin{aligned}
   & \mathcal{RC}_i=b_i\cdot\mathcal{R}\left(\mathbf{w}\right)  \\
   & b_i\ge0  \\
   & w_i\ge0  \\
   & \sum\limits_{i=1}^{n}b_i = 1  \\
   & \sum\limits_{i=1}^{n}w_i = 1  \\
\end{aligned}
\right.
$$
当 $b_i=\frac{1}{n},i=1,\ldots,n$ 时该模型即为风险平价模型, 即各个资产的风险贡献度相同. 

该非线性方程组的求解将转化为如下的优化问题：
$$
\begin{aligned}
 & \underset{\mathbf{w}}{\mathop{\min}}\,f\left(\mathbf{w};\mathbf{b}\right)=\sum\limits_{i=1}^{n}\left(\mathcal{RC}_i-b_i\mathcal{R}(\mathbf{w})\right)^2 \\ 
 & s.t.\ \mathbf{1}^T\cdot\mathbf{w}=1,\mathbf{0}\le\mathbf{w}\le\mathbf{1} \\ 
\end{aligned}
$$
或者：
$$
\begin{aligned}
 & \underset{\mathbf{w}}{\mathop{\min}}\,f\left(\mathbf{w};\mathbf{b}\right)=\sum\limits_{i=1}^{n}\sum\limits_{j=1}^{n}{\left(\frac{\mathcal{RC}_i}{b_i}-\frac{\mathcal{RC}_j}{b_j}\right)^2} \\ 
 & s.t.\ \mathbf{1}^T\cdot\mathbf{w}=1,\mathbf{0}\le\mathbf{w}\le\mathbf{1} \\ 
\end{aligned}
$$
以下主要针对第二个优化问题构建求解方法. 

通过计算目标函数 $f\left(\mathbf{w};\mathbf{b}\right)$ 的梯度和 Hessian 矩阵可以帮助减少计算次数, 尤其在资产数目很大的时候, 目标函数的梯度为：
$$
\begin{aligned}
\frac{\partial f}{\partial w_k} &= \frac{4n}{\sigma^2}w_k\frac{\left(\Sigma \mathbf{w}\right)^2_k}{b_k^2} - \frac{4}{\sigma^2}\frac{\left(\Sigma\mathbf{w}\right)_k}{b_k}\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\\
&+ \frac{4n}{\sigma^2}\sum\limits_{i=1}^{n}w_i^2\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i^2}\Sigma_{ik} - \frac{4}{\sigma^2}\left(\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\right)\cdot\left(\sum\limits_{i=1}^{n}\frac{w_i}{b_i}\Sigma_{ik}\right)\\
&- \frac{4n}{\sigma^4}\left(\sum\limits_{i=1}^{n}w_i^2\frac{\left(\Sigma\mathbf{w}\right)^2_i}{b_i^2}\right)\cdot\left(\sum\limits_{i=1}^{n}w_i\Sigma_{ik}\right) + \frac{4}{\sigma^4}\left(\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\right)^2\cdot\left(\sum\limits_{i=1}^{n}w_i\Sigma_{ik}\right)\\
\end{aligned}
$$
矩阵形式为：（矩阵之间对应元素相乘记为 $.*$, 矩阵的乘方也是各个元素分别乘方）
$$
\begin{aligned}
\frac{\partial f}{\partial\mathbf{w}} &= \frac{4n}{\sigma^2}w.*\left(\Sigma\mathbf{w}\right)^2.*\mathbf{b}^{-2} - \frac{4}{\sigma^2}\left(\mathbf{w}^T\left((\Sigma\mathbf{w}).*\mathbf{b}^{-1}\right)\right)\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\\
&+ \frac{4n}{\sigma^2}\Sigma\cdot\left(\mathbf{w}^2.*\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-2}\right) - \frac{4}{\sigma^2}\left(\mathbf{w}^T\left((\Sigma\mathbf{w}).*\mathbf{b}^{-1}\right)\right)\cdot\Sigma\cdot\left(\mathbf{w}.*\mathbf{b}^{-1}\right)\\
&- \frac{4n}{\sigma^4}\left(\mathbf{w}^2\right)^T\cdot\left((\Sigma\mathbf{w})^2.*\mathbf{b}^{-2}\right)\cdot\Sigma\mathbf{w} + \frac{4}{\sigma^4}\left(\mathbf{w}^T\left((\Sigma\mathbf{w}).*\mathbf{b}^{-1}\right)\right)^2\cdot\Sigma\mathbf{w}\\
\end{aligned}
$$
目标函数的 Hessian 矩阵为：
$$
\begin{aligned}
\frac{\partial^2f}{\partial w_k\partial w_l} &= -\frac{2}{\sigma^2}\frac{\partial f}{\partial w_k}\left(\Sigma\mathbf{w}\right)_l + \frac{4n}{\sigma^2}\left[\delta_{kl}\frac{\left(\Sigma\mathbf{w}\right)^2_k}{b_k^2}+2w_k\frac{\left(\Sigma\mathbf{w}\right)_k}{b_k^2}\Sigma_{kl}\right]\\
&- \frac{4}{\sigma^2}\left[\frac{\left(\Sigma\mathbf{w}\right)_l}{b_l}+\sum\limits_{i=1}^{n}\frac{w_i}{b_i}\Sigma_{il}\right]\frac{\left(\Sigma\mathbf{w}\right)_k}{b_k} - \frac{4}{\sigma^2}\left(\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\right)\frac{\Sigma_{kl}}{b_k}\\
&+ \frac{4n}{\sigma^2}\left[2w_l\frac{\left(\Sigma\mathbf{w}\right)_l}{b_l^2}\Sigma_{lk}+\sum\limits_{i=1}^{n}\frac{w^2_i}{b_i^2}\Sigma_{ik}\Sigma_{il}\right]-\frac{4}{\sigma^2}\left[\frac{\left(\Sigma\mathbf{w}\right)_l}{b_l}+\sum\limits_{i=1}^{n}\frac{w_i}{b_i}\Sigma_{il}\right]\sum\limits_{i=1}^{n}\frac{w_i}{b_i}\Sigma_{ik}\\
&- \frac{4}{\sigma^2}\left(\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\right)\frac{\Sigma_{lk}}{b_l} - \frac{8n}{\sigma^4}\left[w_l\frac{\left(\Sigma\mathbf{w}\right)_l^2}{b^2_l}+\sum\limits_{i=1}^{n}w_i^2\frac{\left(\Sigma\mathbf{w}\right)_i}{b^2_i}\Sigma_{il}\right]\left(\sum\limits_{i=1}^{n}w_i\Sigma_{ik}\right)\\
&- \frac{4n}{\sigma^4}\left(\sum\limits_{i=1}^{n}w_i^2\frac{\left(\Sigma\mathbf{w}\right)^2_i}{b^2_i}\right)\Sigma_{lk} + \frac{8n}{\sigma^6}\left(\Sigma\mathbf{w}\right)_l\left(\sum\limits_{i=1}^{n}w_i^2\frac{\left(\Sigma\mathbf{w}\right)_i^2}{b_i^2}\right)\left(\sum\limits_{i=1}^{n}w_i\Sigma_{ik}\right)\\
&+ \frac{8}{\sigma^4}\left(\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\right)\left[\frac{\left(\Sigma\mathbf{w}\right)_l}{b_l}+\sum\limits_{i=1}^{n}\frac{w_i}{b_i}\Sigma_{il}\right]\cdot\sum\limits_{i=1}^{n}w_i\Sigma_{ik}\\
&+ \frac{4}{\sigma^4}\left(\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\right)^2\cdot\Sigma_{lk} - \frac{8}{\sigma^6}\left(\Sigma\mathbf{w}\right)_l\left(\sum\limits_{i=1}^{n}w_i\frac{\left(\Sigma\mathbf{w}\right)_i}{b_i}\right)^2\sum\limits_{i=1}^{n}w_i\Sigma_{ik}\\
\end{aligned}
$$
其中, $\delta_{kl}$ 为 Kronecker 函数, 即：
$$
\delta_{kl} = \left\{\begin{matrix}
 1,k=l  \\
 0,k\ne l  \\
 \end{matrix} \right.
$$
矩阵形式为：（矩阵之间对应元素相乘记为 $.*$, 矩阵的乘方也是各个元素分别乘方）
$$
\begin{aligned}
\frac{\partial^2f}{\partial\mathbf{w}^2} &= -\frac{2}{\sigma^2}\frac{\partial f}{\partial w}\cdot \left(\Sigma\mathbf{w}\right)^T + \frac{4n}{\sigma^2}\operatorname{D}\left(\left(\Sigma\mathbf{w}\right)^2.*\mathbf{b}^{-2}\right)\\
&+ \frac{8n}{\sigma^2}\operatorname{D}\left(\mathbf{w}.*\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-2}\right)\cdot\Sigma - \frac{4}{\sigma^2}\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)^T\\
&- \frac{4}{\sigma^2}\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\left(\Sigma\cdot\left(\mathbf{w}.*\mathbf{b}^{-1}\right)\right)^T - \frac{4}{\sigma^2}\left(\mathbf{w}^T\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\right)\operatorname{D}\left(\mathbf{b}^{-1}\right)\cdot\Sigma\\
&+ \frac{8n}{\sigma^2}\Sigma\cdot\operatorname{D}\left(\mathbf{w}.*\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-2}\right) + \frac{4n}{\sigma^2}\Sigma\cdot\operatorname{D}\left(\mathbf{w}^2.*\mathbf{b}^{-2}\right)\cdot\Sigma\\
&- \frac{4}{\sigma^2}\left(\Sigma\cdot\left(\mathbf{w}.*\mathbf{b}^{-1}\right)\right)\cdot\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)^T - \frac{4}{\sigma^2}\left(\Sigma\left(\mathbf{w}.*\mathbf{b}^{-1}\right)\right)\cdot\left(\Sigma\left(\mathbf{w}.*\mathbf{b}^{-1}\right)\right)^T\\
&- \frac{4}{\sigma^2}\left(\mathbf{w}^T\cdot\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\right)\Sigma\cdot\operatorname{D}\left(\mathbf{b}^{-1}\right) - \frac{8n}{\sigma^4}\left(\Sigma\mathbf{w}\right)\cdot\left(\mathbf{w}.*\left(\Sigma\mathbf{w}\right)^2.*\mathbf{b}^{-2}\right)^T\\
&- \frac{8n}{\sigma^4}\left(\Sigma\mathbf{w}\right)\left(\Sigma\left(\mathbf{w}^2.*\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-2}\right)\right)^T - \frac{4N}{\sigma^4}\left(\left(\mathbf{w}^2\right)^T\cdot\left(\left(\Sigma\mathbf{w}\right)^2.*\mathbf{b}^{-2}\right)\right)\cdot\Sigma\\
&+ \frac{8n}{\sigma^6}\left(\left(\mathbf{w}^2\right)^T\cdot\left(\left(\Sigma\mathbf{w}\right)^2.*\mathbf{b}^{-2}\right)\right)\left(\Sigma\mathbf{w}\right)\cdot\left(\Sigma\mathbf{w}\right)^T\\
&+ \frac{8}{\sigma^4}\left(\mathbf{w}^T\cdot\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\right)\left(\Sigma\mathbf{w}\right)\cdot\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)^T\\
&+ \frac{8}{\sigma^4}\left(\mathbf{w}^T\cdot\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\right)\left(\Sigma\mathbf{w}\right)\cdot\left(\Sigma\cdot\left(\mathbf{w}.*\mathbf{b}^{-1}\right)\right)^T\\
&+ \frac{4}{\sigma^4}\left(\mathbf{w}^T\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\right)^2\Sigma - \frac{8}{\sigma^6}\left(\mathbf{w}^T\cdot\left(\left(\Sigma\mathbf{w}\right).*\mathbf{b}^{-1}\right)\right)^2\cdot\left(\Sigma\mathbf{w}\right)\cdot\left(\Sigma\mathbf{w}\right)^T\\
\end{aligned}
$$
其中, $\operatorname{D}\left(\cdot\right)$ 是对角化函数, 即对于向量 $\mathbf{x}$, 有：
$$
\operatorname{D}\left(\mathbf{x}\right) = \left(\begin{matrix}
{{x}_{1}} & 0 & \cdots  & 0  \\
0 & {{x}_{2}} & \cdots  & 0  \\
\vdots  & \vdots  & \ddots  & \vdots   \\
0 & 0 & \cdots  & {{x}_{n}}  \\
\end{matrix}\right)
$$
对于维度较高的问题, 也可以先求解如下的凸规划问题：
$$
\begin{aligned}
& \underset{\mathbf{x}}{\mathop{\min}}\,\mathcal{R}\left(\mathbf{x}\right)=\sqrt{\mathbf{x}^T\Sigma\mathbf{x}}\\
& s.t.\ \sum\limits_{i=1}^{n}b_i\operatorname{ln}x_i\ge c, \mathbf{0}\le\mathbf{x}\le\mathbf{1}\\
\end{aligned}
$$
其中, $c$ 是满足 $c<\sum\limits_{i=1}^{n}b_i\operatorname{ln}b_i$ 的任意常数, 最后将最优解归一化得到原问题的解：
$$
\mathbf{w} = \frac{\mathbf{x}}{\mathbf{1}^T\mathbf{x}}
$$
Maillard，Roncalli 和 Teïletche（2008）在他们的研究中详细讨论风险平价的性质。首先，组合中波动较高的证券（或者相关性高的证券）在权重计算时会受到惩罚，获得更小的权重。当所有成分的相关系数相同并且 Sharpe 比率也相等时，风险平价组合是 Markowitz 最优的；当组合所有成分证券相关系数相等时，那么风险平价即为波动率倒数加权。最后可以证明，风险平价介于等权重和最小方差之间，其波动大于最小方差，小于等权重组合。

In [2]:
# 组合优化
PC = QS.PortfolioConstructor.CVXPC()

# 设置相关数据
TargetDT = dt.datetime(2018, 7, 31)
FT = HDB.getTable("stock_cn_day_bar_nafilled")
TargetIDs = FT.getID()[:500]
PC.Args["目标ID"] = TargetIDs
PC.Args["预期收益"] = FT.readData(factor_names=["chg_rate"], ids=TargetIDs, dts=[TargetDT]).iloc[0, 0, :]
RT = RDB.getTable("BarraRiskData")
#PC["协方差矩阵"] = RT.readCov(dts=[TargetDT], ids=TargetIDs)[TargetDT]
PC.Args["因子协方差阵"] = RT.readFactorCov(dts=[TargetDT]).loc[TargetDT]
PC.Args["风险因子"] = RT.readFactorData(dts=[TargetDT], ids=TargetIDs).loc[:, TargetDT, :]
PC.Args["特异性风险"] = RT.readSpecificRisk(dts=[TargetDT], ids=TargetIDs).loc[TargetDT, :]
PC.Args["成交金额"] = FT.readData(factor_names=["amount"], ids=TargetIDs, dts=[TargetDT]).iloc[0, 0, :]
PC.Args["初始投资组合"] = pd.Series(0.0,index=TargetIDs)
PC.Args["总财富"] = 1000000000

# 设置优化目标
Objective = QS.PortfolioConstructor.RiskBudgetObjective(pc=PC)
Objective.Args["预算因子"] = "等权"
PC.Args["优化目标"] = Objective

# 设置约束条件
# 预算约束
iConstraint = QS.PortfolioConstructor.BudgetConstraint(pc=PC)
iConstraint.Args["限制上限"] = 1.0
iConstraint.Args["限制下限"] = 1.0
PC.Args["约束条件"].append(iConstraint)
# 权重约束
iConstraint = QS.PortfolioConstructor.WeightConstraint(pc=PC)
iConstraint.Args["限制上限"] = 1.0
iConstraint.Args["限制下限"] = 0.0
PC.Args["约束条件"].append(iConstraint)

# 求解优化问题
Portfolio, ResultInfo = PC.solve()

print(Portfolio)
print(ResultInfo)

000001.SZ    0.002571
000002.SZ    0.002372
000004.SZ    0.002671
000005.SZ    0.001786
000006.SZ    0.001820
               ...   
000985.SZ    0.001501
000987.SZ    0.001682
000988.SZ    0.002169
000989.SZ    0.003248
000990.SZ    0.003701
Length: 450, dtype: float64
{'Status': 1, 'Msg': 'optimal', 'solver_name': 'ECOS', 'solve_time': 7.382363162252526, 'setup_time': 0.06721601494939676, 'num_iters': 26, 'ReleasedConstraint': []}
